In [5]:
import itertools
import collections

import cirq
import numpy as np
import sys
sys.path.append("..")

from sim.circuit import gen_1d_chain_weak_link_random_circuit
from sim.simulate import large_scale_xeb_sim

In [2]:
exponents = np.linspace(0, 7/4, 8)
SINGLE_QUBIT_GATES = tuple(
    cirq.PhasedXZGate(x_exponent=0.5, z_exponent=z, axis_phase_exponent=a)
    for a, z in itertools.product(exponents, repeat=2)
)

MAX_DEPTH = 100


In [10]:
res = {}
N = 8
link_freq = 8
n_circuits = 5
qubits = cirq.LineQubit.range(N)
circuits = [gen_1d_chain_weak_link_random_circuit
    (
        qubits,
        depth=MAX_DEPTH,
        link_frequency=link_freq,
        single_qubit_gates=SINGLE_QUBIT_GATES,
        two_qubit_op_factory=lambda a, b, _: cirq.SQRT_ISWAP(a, b)
    )
    for _ in range(n_circuits)
]

In [14]:
cycle_depths = [18, 26, 32]
res = collections.defaultdict(list)

e_paulis = np.linspace(5e-3, 2.5e-2, 1)
for e_pauli in e_paulis:
    e_dep = e_pauli / (1 - 1/2**2)
    noise_model = cirq.devices.noise_model.ConstantQubitNoiseModel(cirq.depolarize(e_pauli))
    
    xebs = np.mean([
        large_scale_xeb_sim(circuit, noise_model, cycle_depths=cycle_depths)
        for circuit in circuits
    ], axis=0)
    for i, d in enumerate(cycle_depths):
        estimated_fidelity = (1 - e_dep) ** ((2 * d + 1) * N)
        ratio = estimated_fidelity / xebs[i]
        res[d].append(ratio)


8 qubits noisy state simulation: 100%|██████████| 65/65 [00:02<00:00, 26.28it/s] 


In [15]:
res

defaultdict(list,
            {18: [0.19859533014684436],
             26: [0.1888538733423016],
             32: [0.19805309415401806]})